# TEXT CLASSIFICATION USING PYTORCH 

- As most of the users are prefering pytorch over tensorflow and I am also finding pytorch more of like a maths that we learn as basic of machine learning. In tensorflow the beauty of Maths is abstract, So I am here trying to understand/Learn pytorch as much possible as I can. I am mostly working on TEXTUAL data. As my area of focus is NLP. 

- In this tutorial I am gonna build a text classfier using Linear Model. 
- I am also gonna keep updating in future notebooks . 
- I hope this will help anyone who's trying to use/ learn pytorch. 




![PytorchGIF](https://images.ctfassets.net/rc8q7tcpu9y3/7JrqmBOgQcEQWGuKEiSuow/4f2e59badb8cecb518fb38752feebf61/Facebook-PyTorch-Conference-Experience-Design-Brand-Logo-Construction.gif)
source : https://images.ctfassets.net/rc8q7tcpu9y3/7JrqmBOgQcEQWGuKEiSuow/4f2e59badb8cecb518fb38752feebf61/Facebook-PyTorch-Conference-Experience-Design-Brand-Logo-Construction.gif

### What is torch?
The torch package contains data structures for multi-dimensional tensors and defines mathematical operations over these tensors. Additionally, it provides many utilities for efficient serializing of Tensors and arbitrary types, and other useful utilities.(copied from documentation) 

In [ ]:
# importing torch 
# nn is a module in pytorch package which helps and provide us various 
import torch
import torch.nn as nn 
import numpy as np 
import pandas as pd 

# train_test_split
# TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# scipy to convert .. sparse matrix to dense 
import scipy 
# improting optimizers 
from torch import optim

# for ploting loss and train& val prediction accuracy 
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
torch.__version__

## Let's look in data

- here we got the data in two different files. so I used df1 & df2 to import the data 
- then I used pandas concat() function doing axis=0 so the rows will get add from second dataset(df2)
- deleted the df1 and df2 



 <img align=centre src = "https://i.pinimg.com/originals/28/18/74/2818749911c12b7f854d45e250c0b6d1.gif" width="1000" height="700">



In [ ]:
df1 = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
df2 = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
df1['label'] = 1
df2['label'] = 0
df = pd.concat([df1, df2], axis=0)

del df1 
del df2
df.head()

In [ ]:
print(f'Shape of the dataset: {df.shape}')
print(f'\nSum of nulls:\n{df.isna().sum()}')

In [ ]:
df.info()

# Text preprocessing 


In [ ]:
# Text preprocessing 

def normalise_text(text):
    
    text = text.str.lower()
    text = text.str.replace(r"\#", "" )
    text = text.str.replace(r'http\S+', "URL")
    text = text.str.replace(r"@", "")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]"," ")
    text = text.str.replace("\s{2,}"," ")
    
    return text

In [ ]:
df['text']= df['title']+" "+df['text']

In [ ]:
df['text'] = normalise_text(df['text'])
del df['title']
del df['subject']
del df['date']

# Train Test Splitting

In [ ]:
# let's split the test and train 
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3)

In [ ]:
# creating vectors for the sentence 
vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=10000)

# Learn vocabulary from training texts and vectorize training texts.
X_train = vectorizer.fit_transform(X_train)

# Vectorize test texts.
X_test = vectorizer.transform(X_test)

# CONVERTING INTO TENSORS 
- Tensors are a specialized data structure that are very similar to arrays and matrices. In PyTorch, we use tensors to encode the inputs and outputs of a model, as well as the model’s parameters.

- Tensors are similar to NumPy’s ndarrays, except that tensors can run on GPUs or other specialized hardware to accelerate computing. If you’re familiar with ndarrays, you’ll be right at home with the Tensor API. If not, follow along in this quick API walkthrough.

In [ ]:
#X_train = torch.tensor(scipy.sparse.csr_matrix.todense(X_train)).float()
#X_test = torch.tensor(scipy.sparse.csr_matrix.todense(X_test)).float()

X_train = torch.tensor(X_train.todense()).float()
X_test = torch.tensor(X_test.todense()).float()

In [ ]:
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

# Building Model 

In [ ]:
model = nn.Sequential(
                nn.Linear(X_train.shape[1],128 ),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(128, df['label'].nunique()),
                nn.LogSoftmax(dim=1)
)


# Loss, Optimzer

In [ ]:
# defining the loss 
criterion = nn.NLLLoss()

# Forward pass, get our logits
logps = model(X_train)

# Calculate the loss with the logits and the labels
loss = criterion(logps, y_train)


loss.backward()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.002)

# Training Loop

In [ ]:
%%time
train_losses = []
test_losses = []
test_accuracies = []

epochs = 100
for e in range(epochs):
    optimizer.zero_grad()

    output = model.forward(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    train_loss = loss.item()
    train_losses.append(train_loss)
    
    optimizer.step()
    
    
    # Turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        model.eval()
        log_ps = model(X_test)
        test_loss = criterion(log_ps, y_test)
        test_losses.append(test_loss)

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y_test.view(*top_class.shape)
        test_accuracy = torch.mean(equals.float())
        test_accuracies.append(test_accuracy)

    model.train()
    if (e+1)%10==0:
        print(f"Epoch: {e+1}/{epochs}.. ",
              f"Training Loss: {train_loss:.3f}.. ",
              f"Test Loss: {test_loss:.3f}.. ",
              f"Test Accuracy: {test_accuracy:.3f}")

    

# Let's Plot the Learning and Loss Curve

In [ ]:
plt.figure(figsize=(12, 5))
ax = plt.subplot(121)
plt.xlabel('epochs')
plt.ylabel('negative log likelihood loss')
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False);
plt.subplot(122)
plt.xlabel('epochs')
plt.ylabel('test accuracy')
plt.plot(test_accuracies);

# 😎😎DO UPVOTE FOR CHANDLER😎😎 
![Chandler](https://media.tenor.com/images/b7b886de0e04e771361f730414f52919/tenor.gif)